In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tnrange


# Make Neural

In [2]:
raw_train = np.load("/media/hdd01/sklee/CA1_inputs/stim_T10_Ne2000_gA0.6_tauA1_gN0.8_Ni200_gG0.1_gB0.1_Er0.5_Ir7.4_random_NR_rep16_stimseed1.npy")
raw_train = raw_train[raw_train[:,2] <= 10000]


rep = 16
if rep > 0:
    pick_idx = np.arange(1,(rep-1)*2,2)
    start_idx = np.zeros((rep+1))
    raw_start_idx = np.where(np.diff(raw_train[:,2]) <= -9000)[0][pick_idx] + 1
    start_idx[1:-1] += raw_start_idx
    start_idx[-1] = raw_train.shape[0]
    start_idx = start_idx.astype(int)
    print(start_idx)
    for i in range(rep):
        raw_train[start_idx[i]:start_idx[i+1],2] += i*10000

print(raw_train.shape)
raw_train = np.delete(raw_train, np.where(np.isnan(raw_train) == True)[0], 0)
print(raw_train.shape)
E_train = raw_train[np.where(raw_train[:,0]>0)]
I_train = raw_train[np.where(raw_train[:,0]<0)]
print(E_train.shape)
print(I_train.shape)

[     0  26161  52594  78865 104711 130845 157142 183531 209354 235636
 261372 287535 313690 339767 365814 391961 418138]
(418138, 3)
(418138, 3)
(181496, 3)
(236642, 3)


In [3]:
bin_size = 1 # in milliseconds
E_no = int(np.max(raw_train[:,1][np.where(raw_train[:,0] > 0)])) + 1
I_no = int(np.max(raw_train[:,1][np.where(raw_train[:,0] < 0)])) + 1
bin_no = int(int(np.max(raw_train[:,2]) + 1) / bin_size)
E_neural = np.zeros((bin_no, E_no))
I_neural = np.zeros((bin_no, I_no))

print(E_no)
print(I_no)

bin_edges = np.arange(0, bin_no*bin_size + bin_size, bin_size)
print(bin_edges.shape)
print(bin_edges)

E_counts, E_edges = np.histogram(E_train[:,2], bin_edges)
I_counts, I_edges = np.histogram(I_train[:,2], bin_edges)
print(np.sum(E_counts))
print(np.sum(I_counts))
print(raw_train.shape)

E_count = 0
for i in tnrange(E_counts.shape[0]):
    bin_count = E_counts[i]
    for j in range(bin_count):
        syn = int(E_train[E_count,1])
        E_neural[i, syn] += 1
        E_count += 1

I_count = 0
for i in tnrange(I_counts.shape[0]):
    bin_count = I_counts[i]
    for j in range(bin_count):
        syn = int(I_train[I_count,1])
        I_neural[i, syn] += 1
        I_count += 1
        
print(E_neural.shape)
print(I_neural.shape)
print(np.sum(E_neural))
print(np.sum(I_neural))
print(np.sum(E_neural)/E_neural.size)
print(np.sum(I_neural)/I_neural.size)
print(np.max(E_neural))
print(np.max(I_neural))

2000
200
(160002,)
[     0      1      2 ... 159999 160000 160001]
181496
236642
(418138, 3)


<ipython-input-3-837a82b28ea2>:22: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(E_counts.shape[0]):


<ipython-input-3-837a82b28ea2>:30: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(I_counts.shape[0]):



(160001, 2000)
(160001, 200)
181496.0
236642.0
0.0005671714551784052
0.007395016281148243
2.0
2.0


In [4]:
np.save("/media/hdd01/sklee/CA1_inputs/Espikes_T10_Ne2000_gA0.6_tauA1_gN0.8_Ni200_gG0.1_gB0.1_Er0.5_Ir7.4_random_NR_neural.npy", E_neural)
np.save("/media/hdd01/sklee/CA1_inputs/Ispikes_T10_Ne2000_gA0.6_tauA1_gN0.8_Ni200_gG0.1_gB0.1_Er0.5_Ir7.4_random_NR_neural.npy", I_neural)

# Make CA1 Neural